# Задание по теме keras, simple ANN

ДЗ - обучить при помощи tf (keras) API любое ДЗ по обучению классических моделей.

Мы выбираем датасет Walmart.csv и с помощью keras решаем задачу регрессии, то есть по имеющимся данным построим модель, описывающую пятничные продажи в зависимости от набора параметров.

- План работы: <br>
Следует загрузить файл csv датасета.<br>
Произвести предобработку данных (выкинуть строки с пропусками). <br>
Выделить независимые переменные - характеристики (features) модели.<br>
Выделить колонку целевой (target) переменной - суммы пятничных продаж.<br>
Разделить данные на тренировочный и тестовый датасет.<br>
Создать модель нейронной сети с использованием keras.<br>
Обучить нейросеть.<br>
Провести проверку нейросети на тестовой выборке. Рассчитать метрики на тестовой выборке. <br>

Загрузка необходимых библиотек

In [181]:
%reset
# Загрузка необходимых библиотек
import pandas as pd

Nothing done.


Загрузка данных

In [182]:
# Загрузка данных
data = pd.read_csv('Walmart.csv')

# Размер таблицы данных
print(f' Table dimensions: {data.shape}')

# Просмотр первых 15 строк
data.head(7)

 Table dimensions: (6435, 8)


,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,1,05-02-2010,1643690.90,0,42.31,2.572,211.096358,8.106
1,1,12-02-2010,1641957.44,1,38.51,2.548,211.242170,8.106
2,1,19-02-2010,1611968.17,0,39.93,2.514,211.289143,8.106
3,1,26-02-2010,1409727.59,0,46.63,2.561,211.319643,8.106
4,1,05-03-2010,1554806.68,0,46.50,2.625,211.350143,8.106
5,1,12-03-2010,1439541.59,0,57.79,2.667,211.380643,8.106
6,1,19-03-2010,1472515.79,0,54.58,2.720,211.215635,8.106


In [183]:
# Выкидываем пропуски на тот случай, если они есть. Реально их нету :)
# data.dropna()

Среди данных есть колонка с датами. Для построения модели регрессии дата не нужна.
Остальные параметры - номер магазина, совпадение с праздником, температура, стоимость топлива, индекс потребительских цен, уровень безработицы, - являются потенциально важными для модели.

In [184]:
# Удаление колонки с датами
data = data.drop('Date', axis = 1)

Кодирование номера магазина с помощью One Hot Encoder

In [185]:
# One-Hot Encoding
data_encoded = pd.get_dummies(data, columns=['Store'], drop_first=True)

In [186]:
# Выделение целевой переменной
target = data_encoded['Weekly_Sales']  # 
# Выделение признаков
features = data_encoded.drop('Weekly_Sales', axis = 1) # 


Разделение данных на тренировочный и тестовый наборы

In [187]:
from sklearn.model_selection import train_test_split

# Приведение данных к стандартному виду:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
features_std = scaler.fit_transform(features)

# Разделение данных
X_train, X_test, y_train, y_test = train_test_split(features_std, target, test_size=0.2, random_state=42)

Создание модели нейронной сети с использованием Keras

In [188]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Создание модели
model = Sequential()
model.add(Dense(4, activation='relu', input_dim=X_train.shape[1]))
model.add(Dense(8, activation='relu'))
model.add(Dense(1))  # Выходной слой для регрессии

c:\Users\varaksa_yua\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Компиляция модели

In [189]:
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import EarlyStopping

optimizer = Adam(learning_rate = 2.0)
model.compile(optimizer=optimizer, loss='mean_absolute_error', metrics=['mean_absolute_error'])

Добавляем коллбэк для уменьшения скорости обучения по мере обучения и второй коллбэк для остановки обучения при выходе на плато

In [190]:
# Create the ReduceLROnPlateau callback
patience = 25
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor = 0.5, patience = patience, min_lr=0.001)
# Create the EarlyStopping callback
# early_stopping = EarlyStopping(monitor='val_loss', patience = 3 * patience, restore_best_weights=True)

Обучение модели

In [191]:
# Обучение модели
batch_size = X_train.shape[0]
batch_size = 32
model.fit(X_train, y_train, epochs=1500, batch_size = batch_size, validation_split=0.2, callbacks=[reduce_lr])

Epoch 1/1500
129/129 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 530155.9375 - mean_absolute_error: 530155.9375 - val_loss: 119620.9297 - val_mean_absolute_error: 119620.9297 - learning_rate: 2.0000
Epoch 2/1500
129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 119542.4375 - mean_absolute_error: 119542.4375 - val_loss: 122979.3984 - val_mean_absolute_error: 122979.3984 - learning_rate: 2.0000
Epoch 3/1500
129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 111480.2344 - mean_absolute_error: 111480.2344 - val_loss: 107257.9375 - val_mean_absolute_error: 107257.9375 - learning_rate: 2.0000
Epoch 4/1500
129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 104387.0547 - mean_absolute_error: 104387.0547 - val_loss: 96863.5781 - val_mean_absolute_error: 96863.5781 - learning_rate: 2.0000
Epoch 5/1500
129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 102438.8594 - mean_absolute_error: 102438.8594 - val_loss: 115014.0781 - val_mean_absolute_error: 115014.0781 - learning_rate: 2.0000
Epoch 6/1500
129/129 ━

Оценка модели на тестовой выборке

In [192]:
from sklearn.metrics import root_mean_squared_error 
from sklearn.metrics import r2_score

# Предсказание на тренировочной выборке
y_pred = model.predict(X_train)
# Вычисление среднеквадратичной ошибки
rmse = root_mean_squared_error(y_train, y_pred)
print(f'Root Mean Squared Error: {rmse}')
# Вычисление R^2
r2 = r2_score(y_train, y_pred)
print(f'R^2 Score: {r2*100:.2f}')


# Предсказание на тестовой выборке
y_pred = model.predict(X_test)
# Вычисление среднеквадратичной ошибки
rmse = root_mean_squared_error(y_test, y_pred)
print(f'Root Mean Squared Error: {rmse}')
# Вычисление R^2
r2 = r2_score(y_test, y_pred)
print(f'R^2 Score: {r2*100:.2f}')

161/161 ━━━━━━━━━━━━━━━━━━━━ 0s 527us/step
Root Mean Squared Error: 164374.7230018111
R^2 Score: 91.49
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 448us/step
Root Mean Squared Error: 162109.02430146438
R^2 Score: 91.84


In [193]:
from sklearn.metrics import mean_absolute_error 
print(f'Mean Absolute Error: {mean_absolute_error(y_test, y_pred)}')

from sklearn.metrics import mean_absolute_percentage_error 
print(f'Mean Absolute Percentage Error: {mean_absolute_percentage_error(y_test, y_pred) * 100:.2f}')

Mean Absolute Error: 83813.21138354701
Mean Absolute Percentage Error: 7.62


Выводы: <br>
На основе датасета Walmart, описывающего пятничные продажи в 45 магазинах соответствующей сети за период в несколько лет, и с помощью библиотеки keras составлена модель полносвязной нейронной сети, решающая задачу регрессии , позволяющая прогнозировать продажи для каждого из этих магазинов в зависимости от следующих параметров: 
- совпадение даты с праздничными днями
- температура воздуха
- стоимость топлива
- текущий индекс потребительских цен
- текущий уровень безработицы <br>

Использовалось адаптивное уменьшение скорости обучения, начиная с learning rate = 2: <br>
patience = 25 <br>
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor = 0.5, patience = patience, min_lr=0.001) <br>
Количество эпох бралось равным 1500. <br>

До того, как мы догадались закодировать номер магазина с помощью OneHotEncoder, среднеквадратичная ошибка предсказания выручки магазина не опускалась ниже 170 тысяч денежных единиц.
После применения OneHotEncoder были достигнуты следующие значения:

Для сети 4-8-1: <br>
Root Mean Squared Error: 162109.02430146438 <br>
R^2 Score: 91.84 <br>
Mean Absolute Error: 83813.21138354701 <br>
Mean Absolute Percentage Error: 7.62 <br>

Для сети 8-16-1: <br>
Root Mean Squared Error: 157018.68704708788 <br>
R^2 Score: 92.35 <br>
Mean Absolute Error: 78455.91114607615 <br>
Mean Absolute Percentage Error: 6.95 <br>

Для сети 16-32-1: <br>
Root Mean Squared Error: 155567.12286468793 <br>
R^2 Score: 92.49 <br>
Mean Absolute Error: 77925.20207799146 <br>
Mean Absolute Percentage Error: 6.89 <br>

Для сети 32-64-1 достигли: <br>
Root Mean Squared Error: 156089.51902446602 <br>
R^2 Score: 92.44 <br>
Mean Absolute Error: 77383.25268550894 <br>
Mean Absolute Percentage Error: 6.83 <br>

Видим, что структуры полносвязной сети 8-16-1 или 16-32-1 вполне достаточно для данной задачи. Увеличение числа нейронов в два раза не дает значимого улучшения метрик. 

Полученные значения метрик достаточно хорошие, и, судя по информации из интернета, среднеквадратичная ошибка порядка 150 тысяч вполне приличная для данного датасета.

Настораживает одно наблюдение: по факту, хорошие значения для обученной нейросети получались не потому, что в ходе обучения мы приходили от высоких значений метрик к желаемым низким значениям, а потому, что в результате манипуляций с гиперпараметрами модели в ходе последовательных перезапусков мы получали ситуацию, когда уже на первой эпохе расчета метрика для валидационной выборки оказывалась достаточно хорошей (не порядка миллиона для средней абсолютной ошибки MAE (mean_absolute_error), как в "плохих" попытках, а MAE порядка 110-150 тысяч), что и позволяло в ходе обучения прийти к ошибке MAE~80 тысяч на тестовой выборке.
Если же при запуске обучения модели оказывалось, что на первой эпохе для валидационной выборки MAE~1 миллиона, то обучение приводит в лучшем случае к MAE~470 тысяч для валидационной выборки, где и застревает. Это слишком много.

То есть впечатление такое, что в каких-то случаях мы случайным образом оказываемся вблизи глобального минимума в фазовом пространстве параметров модели уже на этапе первоначального задания весов модели, и поэтому этот минимум благополучно находим, а в других случая на этапе первоначального задания весов мы оказываемся вблизи побочного минимума, и в финале, несмотря на все старания, от него не уходим (даже если задавать гигантскую первоначальную скорость обучения, например 10 или 100).